In [5]:
from src.data.datasets import SiteDataset
import jax.numpy as jnp
site_data = SiteDataset("5a0546857ecc773753327266")

floor = site_data.floors[0]
trace = floor.traces[0]
matrices = trace.matrices
position = matrices["position"]
wifi = matrices["wifi"]
import pyro.distributions as dist
from pyro import sample, plate
import pyro
import numpy as np
import torch

In [4]:
height, width = floor.info["map_info"]["height"], floor.info["map_info"]["width"]
floor_uniform2 = dist.Independent(
    dist.Uniform(low=torch.tensor([-0.5, -0.5]), high=torch.tensor([0.5, 0.5])), 1
)


In [5]:
def initial_model(wifi,wifi_is_observed, position=None, pos_is_observed=None, pos_is_not_observed=None):
    #T = position.shape[0]
    T = wifi.shape[0]
    K = wifi.shape[1]
    
    pos_observed = None if position is None else position[pos_is_observed]

    sigma_eps = 0.1/200
    sigma = 0.1/200

    mu_omega_0 = -45/200
    sigma_omega_0 = 5/200
    #sigma_omega = 2/200

    with plate("wifis", K):
        omega_0 = sample("omega_0", dist.Normal(mu_omega_0, sigma_omega_0))
        wifi_locations = sample("wifi_location", floor_uniform2)

    x_pre = sample("x_0", floor_uniform2)

    for t in pyro.markov(range(1, T + 1)):
        # first compute the parameters of the diagonal gaussian distribution p(z_t | z_{t-1})
        x = sample("x_%d" % t, dist.MultivariateNormal(x_pre, sigma*torch.eye(2)))
        
        if(pos_is_observed[t]):
            x_hat = sample("x_hat_%d" %t,fn=dist.Normal(loc=x, scale=sigma_eps),obs=pos_observed[t])
        
        x_pre = x

    distance = torch.cdist(wifi_locations, x)
    signal_strength= omega_0 + 2*jnp.log(distance)
    omega = sample(
            name="omega",
            fn=dist.Normal(loc=signal_strength[wifi_is_observed], scale=sigma_omega),
            obs=wifi[wifi_is_observed]
    )
    
with handlers.seed(rng_seed=0):
    print(initial_model(wifi, position))

NameError: name 'pyro' is not defined